In [1]:
import vk_api
import time
from functools import reduce
from operator import add
import pandas as pd
import numpy
import pandas as pd
import matplotlib as plt
import numpy

In [2]:
m = 0 #счетчик
MessageHistory = []
MenAndWomenMessage = []
Sex = []

In [3]:
def captcha_handler(captcha):
    key = input("Enter Captcha {0}: ".format(captcha.get_url())).strip()
    return captcha.try_again(key)

In [4]:
login, password = '89269416338', '4ADA259292UnserBesterFreundHitlerкрмбл2002'
vk_session = vk_api.VkApi(login, password, captcha_handler=captcha_handler)
try:
    vk_session.authorization()
except vk_api.AuthorizationError as error_msg:
    print(error_msg)
vk = vk_session.get_api()
FriendIdList = vk.friends.get()

In [5]:
m = 0
for _id in FriendIdList['items']:                 
    try:
        time.sleep(0.1); m += 1; print(m, end = ' ')
        MessageHistory.append(vk.messages.getHistory(user_id=_id, count = 200))
    except vk_api.vk_api.ApiError:
        pass

1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71 72 73 74 75 76 77 78 79 80 81 82 83 84 85 86 87 88 89 90 91 92 93 94 95 96 97 98 99 100 101 102 103 104 105 106 107 108 109 110 111 112 113 114 115 116 117 118 119 120 121 122 123 124 125 126 127 128 129 130 131 132 133 134 135 136 137 138 139 140 141 142 143 144 145 146 147 148 149 150 151 152 153 154 155 156 157 158 159 160 161 162 163 164 165 166 167 

In [6]:
MessageChunks = list(map(lambda msg: msg["items"],MessageHistory))

In [7]:
all_messages =  list(reduce(add, MessageChunks))

In [8]:
message_uid = list(map(lambda msg: msg['user_id'],all_messages))
unique_ids = set(message_uid)

In [9]:
uid_to_gender = {uid:vk.users.get(fields = ['sex'], user_ids = uid )[0]['sex'] for uid in unique_ids}

In [10]:
for msg in all_messages:
    msg['user_sex'] = uid_to_gender[msg['user_id']]

In [11]:
for message in all_messages:
    if message['from_id'] == 174050430:
        MenAndWomenMessage.append(message['body'])
        Sex.append(message['user_sex'] - 1)

In [12]:
df = pd.DataFrame()

In [14]:
df['Sex'] = Sex
df['Message'] = MenAndWomenMessage

In [15]:
Message = []
Sex = []
cnt = 0
cntwomen = (len(df.Sex) - numpy.count_nonzero(df.Sex))
for i in range(len(df.Sex.values)):
    if df.Sex[i] == 0:
        Message.append(df.Message[i])
        Sex.append(df.Sex[i])
    else:
        if cnt < cntwomen:
            Message.append(df.Message[i])
            Sex.append(df.Sex[i])
            cnt += 1

In [16]:
datas = pd.DataFrame()

In [17]:
datas['Sex'] = Sex
datas['Message'] = Message

In [24]:
def extract_features(data):
    features = []
    dataLower = data.lower()
    Rualph = 'абвгдеёжзийклмнопрстуфхцчшщъыьэюяabcdefghijklmnopqrstuvwxyz'
    for j in range(len(Rualph)):
        for i in range(len(Rualph)):
            features.append(dataLower.count(Rualph[j] + Rualph[i] + Rualph[k]))
    return features

In [25]:
X = []
Y = []
for data, sex in zip(datas.Message, datas.Sex):
    try:
        X.append(extract_features(data))
    except Exception as error:
        continue
    Y.append(sex)
X = numpy.array(X)
Y = numpy.array(Y)

In [26]:
from sklearn import cross_validation
x_train, x_test, y_train, y_test = cross_validation.train_test_split(X, Y)

In [27]:
from sklearn.ensemble import RandomForestClassifier
model = RandomForestClassifier(n_estimators=1000,  n_jobs=-1, verbose=1)

In [28]:
model.fit(x_train, y_train)

C:\Anaconda3\lib\site-packages\sklearn\utils\validation.py:386: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and willraise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  DeprecationWarning)


ValueError: Found array with 0 feature(s) (shape=(1, 0)) while a minimum of 1 is required.

In [23]:
from sklearn.metrics import accuracy_score
score = accuracy_score(y_test, model.predict(x_test))
print(round(score, 2))

[Parallel(n_jobs=4)]: Done  42 tasks      | elapsed:    0.0s
[Parallel(n_jobs=4)]: Done 192 tasks      | elapsed:    0.1s
[Parallel(n_jobs=4)]: Done 442 tasks      | elapsed:    0.4s
[Parallel(n_jobs=4)]: Done 792 tasks      | elapsed:    0.7s
[Parallel(n_jobs=4)]: Done 1000 out of 1000 | elapsed:    0.9s finished


0.56


In [167]:
if numpy.count_nonzero(y_test) > (len(y_test) - numpy.count_nonzero(result)):
    print('Men')
else:
    print('Women')

Men
